In [1]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=52676056147fe5bd7b9cd5dca424dacfa34ccb890069c66d25053bd50bce6d79
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


# Regression Evaluation

## Linear Regression Example Code *Along*

This notebook is the reference for the video lecture on the Linear Regression Code Along. Basically what we do here is examine a dataset with Ecommerce Customer Data for a company's website and mobile app. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

First thing to do is start a Spark Session

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark= SparkSession.builder.appName('lr_example').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [6]:
# Use Spark to read in the Ecommerce Customers csv file.
data= spark.read.csv('/content/sample_data/Ecommerce_Customers.csv',inferSchema=True,header=True)

In [7]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [8]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [9]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [10]:
for item in data.head():
  print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


##Setting Up DataFrame for Machine Learning

In [11]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [12]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [13]:
assembler=VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],outputCol='features')

In [14]:
output=assembler.transform(data)

In [15]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [16]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [17]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [18]:
final_data=output.select('features','Yearly Amount Spent')

In [19]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [20]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [21]:
train_data.describe().show()
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                356|
|   mean|   500.598852299393|
| stddev|   79.7282183476815|
|    min|  275.9184206503857|
|    max|  765.5184619388373|
+-------+-------------------+

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                144|
|   mean|  496.1376924354967|
| stddev|    78.468599590072|
|    min| 256.67058229005585|
|    max|  700.9170916173961|
+-------+-------------------+



In [22]:
# Create a Linear Regression Model object
lr=LinearRegression(labelCol='Yearly Amount Spent')

In [23]:
# Fit the model to the data and call this model lrModel
lr_model=lr.fit(train_data)

In [24]:
test_results=lr_model.evaluate(test_data)

In [27]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} \n Intercept: {}".format(lr_model.coefficients,lr_model.intercept))
# Interesting results....
test_results.residuals.show()

Coefficients: [25.29468558137227,39.19735055223208,-0.07069263832050862,61.760699022790654] 
 Intercept: -1023.9753673139802
+-------------------+
|          residuals|
+-------------------+
| -13.65814480964832|
|-1.5174269944251932|
|-18.342246781853646|
| -6.273550592428421|
| -8.545559483429258|
|-14.802822175418726|
|-24.208356595169107|
| -9.713744084975701|
| -2.861513684949159|
| 2.6257383806027974|
| -6.131111119333923|
| -7.903123900595119|
|-5.9947669617669135|
|-20.308414792188728|
| -16.60499318117388|
| -3.881340652560766|
|  5.166762100981089|
| -2.898536259457444|
| -8.583248998356169|
|-10.620443243812417|
+-------------------+
only showing top 20 rows



In [28]:
test_results.rootMeanSquaredError

10.752463657519947

In [29]:
test_results.r2

0.9810917812747366

In [30]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [31]:
unlabeled_data=test_data.select('features')

In [32]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.5743636841713...|
|[30.8162006488763...|
|[30.8794843441274...|
|[31.0613251567161...|
|[31.0662181616375...|
|[31.1239743499119...|
|[31.1280900496166...|
|[31.2606468698795...|
|[31.3662121671876...|
|[31.4252268808548...|
|[31.4474464941278...|
|[31.5147378578019...|
|[31.5702008293202...|
|[31.5741380228732...|
|[31.5761319713222...|
|[31.6548096756927...|
|[31.7216523605090...|
|[31.7242025238451...|
|[31.8279790554652...|
+--------------------+
only showing top 20 rows



In [33]:
predictions=lr_model.transform(unlabeled_data)

In [34]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...| 333.5870146128419|
|[30.5743636841713...|443.58184075249085|
|[30.8162006488763...|284.42858773032265|
|[30.8794843441274...| 496.4801505772831|
|[31.0613251567161...|496.10101754133086|
|[31.0662181616375...| 463.7361153830931|
|[31.1239743499119...| 511.1554104349349|
|[31.1280900496166...| 566.9664308320304|
|[31.2606468698795...|424.18814494190053|
|[31.3662121671876...| 427.9631441758821|
|[31.4252268808548...| 536.8978297740958|
|[31.4474464941278...|426.50586599581914|
|[31.5147378578019...| 495.8072549582283|
|[31.5702008293202...| 566.2539069335936|
|[31.5741380228732...| 561.0142653417607|
|[31.5761319713222...| 545.1079246418891|
|[31.6548096756927...| 470.0966616265674|
|[31.7216523605090...| 350.6754628913301|
|[31.7242025238451...|511.97113628631666|
|[31.8279790554652...|450.62319079075394|
+--------------------+------------

In [35]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 10.752463657519947
MSE: 115.61547470628723
